#### Import packages

In [1]:

import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.stats.proportion import proportions_ztest
import numpy as np


%matplotlib inline

### Multivariate Test Data - log file  

**Control:**  
● Landing Page: "Book Flight" button.  
● Cost Presentation: Explicitly stating that the cost includes the tip.  
**Experiment 1:**  
● Landing Page: Replaces the "Book Flight" button with a "Fly Now" button.  
● Cost Presentation: Explicitly stating that the cost includes the tip.  
**Experiment 2:**   
● Landing Page: Similar to the control group, this variant retains the "Book Flight" 
button.  
● Cost Presentation: Specifies that the cost does not include the tip.  
**Experiment 3:**  
● Landing Page: Like Experiment 1, this variant includes the "Fly Now" button.  
● Cost Presentation: Similar to Experiment 2, specifies that the tip isn’t included.  

In [2]:
# load flyber log data

log = pd.read_csv('data\Flyber_event_logs.csv')
log['event_time'] = pd.to_datetime(log['event_time'])
log['event_dt'] = log['event_time'].dt.date


In [3]:
# view dataset

log.head()

,event_uuid,user_uuid,event_time,age,session_uuid,experiment_group,user_neighborhood,event_type,event_dt
0,b9e9489e-1218-4715-b42e-604e3745db07,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:34:23,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,open,2019-10-06
1,a4ff8e2c-2701-4a32-9029-bdf6efc155c5,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:36:45,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,#_of_users,2019-10-06
2,41b4f6b1-cc14-4c77-a520-50c1cb9cd592,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:37:31,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,search,2019-10-06
3,73b945e9-691e-4e63-aa4d-da2f74599388,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-12 06:27:13,40-49,8a08005b-7e74-4664-b25d-9e29d8316d71,experiment_1,Manhattan,open,2019-10-12
4,a070700c-8bd9-4b5a-9325-e029fda604c7,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-12 06:28:10,40-49,8a08005b-7e74-4664-b25d-9e29d8316d71,experiment_1,Manhattan,#_of_users,2019-10-12


In [4]:

log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367083 entries, 0 to 367082
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   event_uuid         367083 non-null  object        
 1   user_uuid          367083 non-null  object        
 2   event_time         367083 non-null  datetime64[ns]
 3   age                367083 non-null  object        
 4   session_uuid       367083 non-null  object        
 5   experiment_group   367083 non-null  object        
 6   user_neighborhood  367083 non-null  object        
 7   event_type         367083 non-null  object        
 8   event_dt           367083 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 25.2+ MB


#### Inspect Data

In [5]:

# Unique Events by Experiment Group

log.groupby('experiment_group')['event_uuid'].nunique()


experiment_group
control         91479
experiment_1    91488
experiment_2    92402
experiment_3    91714
Name: event_uuid, dtype: int64

In [6]:
# event dates
list(log.groupby('experiment_group')['event_dt'].unique())

[array([datetime.date(2019, 10, 9), datetime.date(2019, 10, 6),
        datetime.date(2019, 10, 5), datetime.date(2019, 10, 8),
        datetime.date(2019, 10, 11), datetime.date(2019, 10, 7),
        datetime.date(2019, 10, 10), datetime.date(2019, 10, 12)],
       dtype=object),
 array([datetime.date(2019, 10, 6), datetime.date(2019, 10, 12),
        datetime.date(2019, 10, 10), datetime.date(2019, 10, 11),
        datetime.date(2019, 10, 8), datetime.date(2019, 10, 7),
        datetime.date(2019, 10, 9), datetime.date(2019, 10, 5)],
       dtype=object),
 array([datetime.date(2019, 10, 10), datetime.date(2019, 10, 6),
        datetime.date(2019, 10, 7), datetime.date(2019, 10, 5),
        datetime.date(2019, 10, 12), datetime.date(2019, 10, 11),
        datetime.date(2019, 10, 8), datetime.date(2019, 10, 9)],
       dtype=object),
 array([datetime.date(2019, 10, 8), datetime.date(2019, 10, 11),
        datetime.date(2019, 10, 9), datetime.date(2019, 10, 6),
        datetime.date(201

In [7]:
log.groupby('experiment_group')['user_neighborhood'].value_counts()

experiment_group  user_neighborhood
control           Manhattan            64068
                  Brooklyn             18362
                  Queens                4547
                  Bronx                 2683
                  Staten Island         1819
experiment_1      Manhattan            64048
                  Brooklyn             18574
                  Queens                4312
                  Bronx                 2748
                  Staten Island         1806
experiment_2      Manhattan            64636
                  Brooklyn             18687
                  Queens                4653
                  Bronx                 2723
                  Staten Island         1703
experiment_3      Manhattan            64507
                  Brooklyn             18257
                  Queens                4576
                  Bronx                 2648
                  Staten Island         1726
Name: count, dtype: int64

In [8]:

log.groupby('experiment_group')['event_type'].value_counts()

experiment_group  event_type
control           open          56390
                  #_of_users    23612
                  search        11323
                  begin_ride      154
experiment_1      open          56390
                  #_of_users    23626
                  search        11300
                  begin_ride      172
experiment_2      open          56688
                  #_of_users    23953
                  search        11581
                  begin_ride      180
experiment_3      open          56687
                  #_of_users    23557
                  search        11299
                  begin_ride      171
Name: count, dtype: int64

In [9]:
log.groupby('experiment_group')['age'].value_counts()

experiment_group  age  
control           50+      44196
                  40-49    23363
                  18-29    14350
                  30-39     9570
experiment_1      50+      43734
                  40-49    24036
                  18-29    14083
                  30-39     9635
experiment_2      50+      44380
                  40-49    23768
                  18-29    14613
                  30-39     9641
experiment_3      50+      43885
                  40-49    24001
                  18-29    14318
                  30-39     9510
Name: count, dtype: int64

### Multivariate Experiment

In [10]:

log.head()

,event_uuid,user_uuid,event_time,age,session_uuid,experiment_group,user_neighborhood,event_type,event_dt
0,b9e9489e-1218-4715-b42e-604e3745db07,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:34:23,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,open,2019-10-06
1,a4ff8e2c-2701-4a32-9029-bdf6efc155c5,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:36:45,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,#_of_users,2019-10-06
2,41b4f6b1-cc14-4c77-a520-50c1cb9cd592,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-06 05:37:31,40-49,2d7b8da3-573a-447f-9964-acf32526b1db,experiment_1,Manhattan,search,2019-10-06
3,73b945e9-691e-4e63-aa4d-da2f74599388,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-12 06:27:13,40-49,8a08005b-7e74-4664-b25d-9e29d8316d71,experiment_1,Manhattan,open,2019-10-12
4,a070700c-8bd9-4b5a-9325-e029fda604c7,8191a8b8-9445-4661-9500-2bed275576f6,2019-10-12 06:28:10,40-49,8a08005b-7e74-4664-b25d-9e29d8316d71,experiment_1,Manhattan,#_of_users,2019-10-12


In [11]:

event_logs = log.groupby(['event_dt','experiment_group','event_type','user_neighborhood', 'age']).agg(
    {
     'event_uuid': 'count'
     }
)
event_logs.columns = ['event_clicks']
event_logs.reset_index(inplace=True)
event_logs.fillna(0, inplace=True)


In [12]:

event_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2179 entries, 0 to 2178
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   event_dt           2179 non-null   object
 1   experiment_group   2179 non-null   object
 2   event_type         2179 non-null   object
 3   user_neighborhood  2179 non-null   object
 4   age                2179 non-null   object
 5   event_clicks       2179 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 102.3+ KB


In [13]:

event_logs.head(10)

,event_dt,experiment_group,event_type,user_neighborhood,age,event_clicks
0,2019-10-05,control,#_of_users,Bronx,18-29,6
1,2019-10-05,control,#_of_users,Bronx,30-39,6
2,2019-10-05,control,#_of_users,Bronx,40-49,11
3,2019-10-05,control,#_of_users,Bronx,50+,29
4,2019-10-05,control,#_of_users,Brooklyn,18-29,64
5,2019-10-05,control,#_of_users,Brooklyn,30-39,36
6,2019-10-05,control,#_of_users,Brooklyn,40-49,88
7,2019-10-05,control,#_of_users,Brooklyn,50+,173
8,2019-10-05,control,#_of_users,Manhattan,18-29,176
9,2019-10-05,control,#_of_users,Manhattan,30-39,108


In [14]:
# Inspect Count of Unique Events By Experiment Group

event_agg = event_logs.groupby(['experiment_group'])['event_clicks'].sum().reset_index()
event_agg.fillna(0, inplace=True)
event_agg.head()

,experiment_group,event_clicks
0,control,91479
1,experiment_1,91488
2,experiment_2,92402
3,experiment_3,91714


In [15]:

# Summarize Event Type by Experiment Group

event_funnel = event_logs.groupby(['experiment_group','event_type'])['event_clicks'].sum().reset_index()
event_funnel.sort_values(by=['experiment_group','event_clicks'], ascending=False, inplace=True)
event_funnel = event_funnel[event_funnel.event_type != '#_of_users']
event_funnel.head()

,experiment_group,event_type,event_clicks
14,experiment_3,open,56687
15,experiment_3,search,11299
13,experiment_3,begin_ride,171
10,experiment_2,open,56688
11,experiment_2,search,11581


In [17]:

# Display Funnel Chart

fig = px.bar(event_funnel, 
             x="experiment_group", y="event_clicks", 
             color="event_type", 
             color_discrete_sequence=px.colors.qualitative.Set2,         
             barmode='group',
             text_auto=True)
fig.update_layout(
    width=750,
    showlegend=True,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25),
    legend=dict(        
        title='Stage',  # Add a title for the legend
        font = dict(size=10),
        x=0.9,  # Adjust the x position of the legend
        y=0.9,  # Adjust the y position of the legend
        bgcolor='rgba(255,255,255,0.5)'  # Optional: Add background color to the legend (with some transparency)
    )     
)
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title
fig.update_traces(marker_line_color='black',  # Set border color
                  marker_line_width=1.5)  
fig.show()


In [16]:
# Summarize Counts at  Event Types By Experiment Group

event_stats = event_logs.groupby(['experiment_group','event_type'])['event_clicks'].sum().unstack()
event_stats.columns = ['actual_users','booked_rides','opened_app','searched']
event_stats.reset_index(inplace=True)


In [17]:

event_stats

,experiment_group,actual_users,booked_rides,opened_app,searched
0,control,23612,154,56390,11323
1,experiment_1,23626,172,56390,11300
2,experiment_2,23953,180,56688,11581
3,experiment_3,23557,171,56687,11299


In [18]:

# Display Conversion by Experiement Group (booked_rides/opened_app)

# Define a set of bold colors
bold_colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33FB', '#F3FF33']

# Assuming event_stats is your DataFrame
# fig = px.bar(event_stats, x="experiment_group", y=event_stats.booked_rides/event_stats.opened_app, color="experiment_group")

# Updated code
fig = px.bar(event_stats, 
             x="experiment_group", 
             y=event_stats.booked_rides/event_stats.opened_app, 
             text=event_stats.booked_rides/event_stats.opened_app,
             color="experiment_group"
             )

fig.update_traces(texttemplate='%{text:.2%}', textposition='outside')

fig.update_layout(
    width=550,
    height=400,
    plot_bgcolor='white',    
    showlegend=False ,
    margin=dict(l=20, r=20, t=50, b=50),
    titlefont=dict(size=14),
    title={
            'text': "Conversion Rates By Experimental Variant",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },    
)

fig.update_xaxes(
    title_text=" ",
    showgrid=False
)

fig.update_yaxes(
    #title_text="Conversion Rate",
    showgrid=False,
    range=[0, 0.004],
    tickformat=".2%" ,
    tickfont=dict(size=10),
    title_text = "",
    showticklabels=False
)

fig.show()


In [19]:
# Z-Test Comparing Experiment Groups to Control

# Define the alpha level
alpha = 0.05 / 3  # Adjusted alpha for Bonferroni correction

control_obs = event_stats.loc[event_stats["experiment_group"] == "control", "booked_rides"].values[0]
control_cnt = event_stats.loc[event_stats["experiment_group"] == "control", "opened_app"].values[0]


# Perform a test of two proportions for control vs. each experiment group
for group in ["experiment_1", "experiment_2", "experiment_3"]:

    expt_obs = event_stats.loc[event_stats["experiment_group"] == group, "booked_rides"].values[0]
    expt_cnt = event_stats.loc[event_stats["experiment_group"] == group, "opened_app"].values[0]

    ttest_stat, ttest_pval = proportions_ztest(count=np.array([expt_obs, control_obs]), nobs=np.array([expt_cnt,control_cnt]), alternative='two-sided')
    print(f'Control vs {group} two sided z-test: z = {ttest_stat:.4f}, p value = {ttest_pval:.4f}')



Control vs experiment_1 two sided z-test: z = 0.9984, p value = 0.3181
Control vs experiment_2 two sided z-test: z = 1.3765, p value = 0.1687
Control vs experiment_3 two sided z-test: z = 0.8969, p value = 0.3698


### Funnel & Cohort Analyses

In [20]:
user_funnel = log.groupby(['event_type'])['user_uuid'].count().reset_index()
user_funnel.columns  = ['stage', 'users']
user_funnel = user_funnel[user_funnel.stage != '#_of_users']
user_funnel.sort_values(by='users',ascending=False, inplace=True)
user_funnel

,stage,users
2,open,226155
3,search,45503
1,begin_ride,677


In [21]:

fig = px.bar(user_funnel, 
             x="stage", y="users", 
             color="stage", 
             color_discrete_sequence=px.colors.qualitative.Dark24,                
             text_auto=True)
fig.update_layout(
    width=750,
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25)    
)
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title
fig.update_traces(marker_line_color='black',  # Set border color
                  marker_line_width=1.5)  
fig.show()

#### User Segments

In [24]:

demog = log.groupby(['user_neighborhood','age'])['user_uuid'].nunique().reset_index()
demog.head()

,user_neighborhood,age,user_uuid
0,Bronx,18-29,841
1,Bronx,30-39,623
2,Bronx,40-49,1213
3,Bronx,50+,1878
4,Brooklyn,18-29,5767


In [25]:

age = demog.groupby(['age'])['user_uuid'].sum().reset_index()
age.columns = ['Age','Users']
age

,Age,Users
0,18-29,28321
1,30-39,20124
2,40-49,41774
3,50+,64059


In [26]:

borough = demog.groupby(['user_neighborhood'])['user_uuid'].sum().reset_index()
borough.columns = ['Borough', 'Users']
borough

,Borough,Users
0,Bronx,4555
1,Brooklyn,31176
2,Manhattan,107945
3,Queens,7645
4,Staten Island,2957


In [27]:

fig = px.bar(age, 
             x='Age', 
             y='Users',
             width=600,    
             color='Age',
             color_discrete_sequence=px.colors.qualitative.Bold,   
             text_auto=True)
fig.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25)    
    )
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title


fig.show()

In [28]:
fig = px.bar(borough, 
             x='Borough', 
             y='Users',
             width=600,    
             color='Borough',
             color_discrete_sequence=px.colors.qualitative.Dark2,   
             text_auto=True)
fig.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25)    
    )
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title
fig.show()

#### Segment Analysis of Funnel

In [29]:

age_funnel = log.groupby(['age','event_type'])['user_uuid'].count().reset_index()
age_funnel = age_funnel[age_funnel.event_type != '#_of_users']
age_funnel.columns = ['age','stage','users']
age_funnel.sort_values(by=['age','users'], ascending=False, inplace=True)
age_funnel

,age,stage,users
14,50+,open,113261
15,50+,search,15256
13,50+,begin_ride,195
10,40-49,open,56256
11,40-49,search,15090
9,40-49,begin_ride,256
6,30-39,open,22760
7,30-39,search,6019
5,30-39,begin_ride,100
2,18-29,open,33878


In [30]:

fig = px.bar(age_funnel, 
             x="age", y="users", 
             color="stage", 
             color_discrete_sequence=px.colors.qualitative.Safe,   
             barmode='group', 
             text_auto=True)
fig.update_layout(
    width=750,
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25)    
)
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title
fig.update_traces(marker_line_color='black',  # Set border color
                  marker_line_width=1.5)  
fig.show()

In [31]:

bor_funnel = log.groupby(['user_neighborhood','event_type'])['user_uuid'].count().reset_index()
bor_funnel = bor_funnel[bor_funnel.event_type != '#_of_users']
bor_funnel.columns = ['borough','stage','users']
bor_funnel.sort_values(by=['borough','users'], ascending=False, inplace=True)
bor_funnel

,borough,stage,users
18,Staten Island,open,4341
19,Staten Island,search,867
17,Staten Island,begin_ride,14
14,Queens,open,11172
15,Queens,search,2257
13,Queens,begin_ride,31
10,Manhattan,open,158366
11,Manhattan,search,31948
9,Manhattan,begin_ride,474
6,Brooklyn,open,45583


In [32]:

fig = px.bar(bor_funnel, 
             x="borough", y="users", 
             color="stage", 
             color_discrete_sequence=px.colors.qualitative.Safe,   
             barmode='group', 
             text_auto=True)
fig.update_layout(
    width=750,
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(l=25, r=25, t=25, b=25)    
)
fig.update_xaxes(title_text=' ')  # Update x-axis title
fig.update_yaxes(title_text=' ')  # Update y-axis title
fig.update_traces(marker_line_color='black',  # Set border color
                  marker_line_width=1.5)  
fig.show()

### Hypothesis & Next Steps

#### User Interviews

In [34]:
interviews = pd.read_csv('data\Flyber User Interviews.csv')

In [433]:
interviews.head()

,Name,Age,Profession,How Often Do You Use Flyber?,What Do You Use When You Don't Use Flyber?,What do you like about Flyber?,Where are you going when you use Flyber?
0,Fahim Burke,25,Trust Fund Kid,Every couple months,"My family has a private jet, so I only use Fly...",I don't feel like a common driver; I can get a...,"Somewhere out of manhattan, like a cool underg..."
1,Pharrell Campos,22,Influencer,1X a week,Sponsors often pay for me to use their vehicle...,It is super cool to arrive to an art gallery o...,"Like I said, I'm always going to new places an..."
2,Keelan Flores,39,Real Estate,3+ times a week,"If the timing isn't different, I'll take a tax...","I'm in real estate, so I have showings all ove...","To check out a new property, or show a propert..."
3,Kayleigh Mcintosh,42,Plastic Surgeon,1X a month,"Before I started using Flyber, I would use a n...",I like the ability to get home quickly after a...,Usually I take flyber as a treat when I'm exha...
4,Kierran Blackburn,55,Real Estate,1X a month,I just hail a taxi or tell my phone to call a ...,I like being able to go from showing to anothe...,"Generally if a showing runs late, I will pull ..."


In [35]:
interviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Name                                        11 non-null     object
 1   Age                                         11 non-null     int64 
 2   Profession                                  11 non-null     object
 3   How Often Do You Use Flyber?                11 non-null     object
 4   What Do You Use When You Don't Use Flyber?  11 non-null     object
 5   What do you like about Flyber?              11 non-null     object
 6   Where are you going when you use Flyber?    11 non-null     object
dtypes: int64(1), object(6)
memory usage: 748.0+ bytes


In [36]:
pd.set_option('display.width', 500)

In [37]:
interviews

,Name,Age,Profession,How Often Do You Use Flyber?,What Do You Use When You Don't Use Flyber?,What do you like about Flyber?,Where are you going when you use Flyber?
0,Fahim Burke,25,Trust Fund Kid,Every couple months,"My family has a private jet, so I only use Fly...",I don't feel like a common driver; I can get a...,"Somewhere out of manhattan, like a cool underg..."
1,Pharrell Campos,22,Influencer,1X a week,Sponsors often pay for me to use their vehicle...,It is super cool to arrive to an art gallery o...,"Like I said, I'm always going to new places an..."
2,Keelan Flores,39,Real Estate,3+ times a week,"If the timing isn't different, I'll take a tax...","I'm in real estate, so I have showings all ove...","To check out a new property, or show a propert..."
3,Kayleigh Mcintosh,42,Plastic Surgeon,1X a month,"Before I started using Flyber, I would use a n...",I like the ability to get home quickly after a...,Usually I take flyber as a treat when I'm exha...
4,Kierran Blackburn,55,Real Estate,1X a month,I just hail a taxi or tell my phone to call a ...,I like being able to go from showing to anothe...,"Generally if a showing runs late, I will pull ..."
5,Sapphire Dupont,60,Surgeon,2X a month,"Before Flyber, I'd call a taxi service on the ...",I volunteer as a pro-bono surgeon a couple tim...,To the hospital where a patient is located.
6,Tanner Cherry,72,Retired,3+ times a week,Drive my car or call a taxi service.,Flying is so much fun! Plus my wife doesn't li...,All over! It is faster than driving and I don'...
7,Louis Jones,70,Oil Magnate,"5+ times a month, during the 6 months/year I'm...",I have a personal car service on call. My assi...,Flyber gives me the convenience I already have...,Generally between business meetings in the city
8,Robbie Gates,67,"VP, Fortune 50",Tried it once,"Honestly, I thought about using Flyber to supr...",It would be great to use Flyber to go from the...,"The one time I used Flyber, I was going to my ..."
9,Esmae Wagner,65,CEO,Every couple months,I usually use Flyber when I need to get from t...,I like that Flyber saves me time over traffic.,Usually to an important client dinner; Flyber ...


In [38]:
interviews.iloc[:, :4]

,Name,Age,Profession,How Often Do You Use Flyber?
0,Fahim Burke,25,Trust Fund Kid,Every couple months
1,Pharrell Campos,22,Influencer,1X a week
2,Keelan Flores,39,Real Estate,3+ times a week
3,Kayleigh Mcintosh,42,Plastic Surgeon,1X a month
4,Kierran Blackburn,55,Real Estate,1X a month
5,Sapphire Dupont,60,Surgeon,2X a month
6,Tanner Cherry,72,Retired,3+ times a week
7,Louis Jones,70,Oil Magnate,"5+ times a month, during the 6 months/year I'm..."
8,Robbie Gates,67,"VP, Fortune 50",Tried it once
9,Esmae Wagner,65,CEO,Every couple months


In [39]:

list(interviews["What Do You Use When You Don't Use Flyber?"])

['My family has a private jet, so I only use Flyber when I need to get around in the city. Sometimes my friends and I take limosines ..or I have 3 personal cars I can use for joy rides.',
 'Sponsors often pay for me to use their vehicles or set up transportation for me when visiting their restaurants or events.',
 "If the timing isn't different, I'll take a taxi or uber to save money. ",
 'Before I started using Flyber, I would use a normal uber.',
 "I just hail a taxi or tell my phone to call a cab to go to a certain address (I'm always on the phone, so I just use voice commands with my phone most of the time)",
 "Before Flyber, I'd call a taxi service on the phone.",
 'Drive my car or call a taxi service.',
 "I have a personal car service on call. My assistant books Flyber whenever I'd be travelling during peak NYC traffic hours. Time is money and Flyber saves me time! But I let my assistant actually book the Flyber because the first few times I tried booking, the instructions were t

In [40]:
list(interviews.iloc[:, 5])

["I don't feel like a common driver; I can get anywhere in NY in a hurry.",
 'It is super cool to arrive to an art gallery opening or restaurant in a flying car. Everyone always stares. Plus no boring time spent in traffic!',
 "I'm in real estate, so I have showings all over the city. You can imagine I need to travel a lot and traffic can be killer. Flyber allows me to quickly go from location to location without worrying about what time of day it is.",
 'I like the ability to get home quickly after a long day of surgery. ',
 'I like being able to go from showing to another house or my office quickly. With this real estate market you need to act fast!',
 "I volunteer as a pro-bono surgeon a couple times a months. That means I'm on call and might have to get to any hospital. In these cases, time matters. Flyber helps me get to the hospital faster.",
 "Flying is so much fun! Plus my wife doesn't like me to drive in the city anymore. At my age, eyesight starts to go, you know? ",
 'Flyber

In [41]:
list(interviews.iloc[:, 6])

['Somewhere out of manhattan, like a cool underground rave or club opening',
 "Like I said, I'm always going to new places and openings. Galleries, restaurants, clubs, I think I once took Flyber to an 80s themed workout class!",
 "To check out a new property, or show a property. With clients on a very tight schedule looking at some of the best real estate in NY, I'll even call a Flyber to bring us between different properties in my portfolio during one session.",
 "Usually I take flyber as a treat when I'm exhausted and want to get home faster after an especially long day at the hospital.",
 "Generally if a showing runs late, I will pull out my phone and open up the app and call a Flyber. I prefer to use text-to-speech which I can't do with Flyber so it can be a little annoying when I'm already in a hurry.",
 'To the hospital where a patient is located.',
 "All over! It is faster than driving and I don't have to worry about my wife nagging me to be safe.",
 'Generally between business 

In [42]:
#  Age 50 +

list(interviews.iloc[4:,4])

["I just hail a taxi or tell my phone to call a cab to go to a certain address (I'm always on the phone, so I just use voice commands with my phone most of the time)",
 "Before Flyber, I'd call a taxi service on the phone.",
 'Drive my car or call a taxi service.',
 "I have a personal car service on call. My assistant books Flyber whenever I'd be travelling during peak NYC traffic hours. Time is money and Flyber saves me time! But I let my assistant actually book the Flyber because the first few times I tried booking, the instructions were too small.",
 'Honestly, I thought about using Flyber to suprise my grandson or granddaughter with a visit to one of their sporting games. Luckily my daughter was around to help me book the ride. I usually just use Uber because it remembers my addresses and has all my favorite places saved, so I guess I always just open that up since it is so convenient and saves me time. Though now that I say that, I really should use Flyber again since it would sav